<center><h1> <strong>Reduction of the dimensionality of a data set <br/> based on entropy   </strong>
 </h1>

<center><h2>Distributed systems  </h2>
<center><h3>Universidad Nacional de Colombia</h3>
<center><h4>Octuber 2022</h4>

-------
-------

by: 

Javier Eduardo Duarte Aunta

-------
-------

## 0. Make data base

In [19]:
from asyncore import write
import random
import string

def ramdomLine (NColum):

    line=random.choice(string.ascii_letters)
    for i in range(NColum-1):
        line=line+',' + random.choice(string.ascii_letters)

    return line

def makeCSVFile (fileName, Nrows, NColum):
    
    f = open(f"{fileName}", "w")
    for i in range(Nrows):
        f.write(ramdomLine(NColum)+"\n")

    f.close()

Nrows = 2560
NColum = 10
makeCSVFile("matrix.txt",Nrows=Nrows,NColum=NColum)

print(f"Created a test file with:\n > rows: {Nrows} \n > columns: {NColum}")

Created a test file with:
 > rows: 2560 
 > columns: 10


## 1. Sequential code

In [20]:
#@title main_sequential.c
%%writefile main_sequential.c

/*******************************************************************************
*
* Program: Dimensionality Reduction With Entropy to a data Matrix set
* 
* Description: Project - Distributed Computing
* 
* Universidad Nacional de Colombia
* 
* Author: Javier Eduardo Duarte Aunta
*
* Octuber 2022
*******************************************************************************/

#include <stdio.h>
#include <math.h>
#include <string.h>
#include <sys/time.h>

#define MATRIX_ROW 2560
#define MATRIX_COLUNM 10
#define STRING_LENGTH 4
#define MAX_LEN_LINE STRING_LENGTH*MATRIX_COLUNM

#define PRINT_LOGS 0
#define PRINT_LOGS_MATRIX 0

//Matrix type double (*)[MATRIX_ROW]
double similarity_matrix[MATRIX_ROW][MATRIX_ROW];

//Matrix type char (*)[MATRIX_COLUNM][STRING_LENGTH]
char main_matrix[MATRIX_ROW][MATRIX_COLUNM][STRING_LENGTH]={/*  */};


// function to calculate Hamming distance
int hammingDist(char* str1, char* str2)
{
    int i = 0, count = 0;
    while (str1[i] != '\0') {
        if (str1[i] == str2[i] && str1[i] != '\n' && str2[i] != '\n')
            count++;
        i++;
    }
    return count;
}

void printDataMatrix(char data_matrix[][MATRIX_COLUNM][STRING_LENGTH]){
    // Print read data
    printf("Printing read matrix:\n");
    for(int i=0; i<MATRIX_ROW; i++){
        for (int j=0; j<MATRIX_COLUNM; j++){
            printf("%s ", data_matrix[i][j]);
        }
        //printf(" %d",i);
        printf("\n");
    }
}

void readCSVFile (char main_matrix[MATRIX_ROW][MATRIX_COLUNM][STRING_LENGTH]){
    
    if (PRINT_LOGS){
    printf("-----------------------------------------\n");
    printf("BEGIN - function: readCSVFile \n");
    printf("-----------------------------------------\n");
    }

   char line[MAX_LEN_LINE];
   char *token;

    FILE *file;
    file = fopen("matrix.txt", "r");


    if (file == NULL){
        printf("Error opening file.\n");
    }

    for (int row = 0; row < MATRIX_ROW; row++){
        fgets(line, MAX_LEN_LINE, file);
        //printf("%s", line);

        token= strtok(line, ",");
        for (int colunm = 0; colunm < MATRIX_COLUNM; colunm++)
        {
            if (colunm == MATRIX_COLUNM-1){
                token= strtok(token, "\n");
                strcpy(main_matrix[row][colunm],token);
            }
            else{
                strcpy(main_matrix[row][colunm],token);
                token= strtok(NULL, ",");
            }

        }
        
    }
    fclose(file);
    if (PRINT_LOGS_MATRIX)
        printDataMatrix(main_matrix);

    if (PRINT_LOGS){   
    printf("-----------------------------------------\n");
    printf("END - function: readCSVFile \n");
    printf("-----------------------------------------\n");
    }
}

void printSimilarityMatrix( double similarity_matrix[MATRIX_ROW][MATRIX_ROW]){

    // Print similarity matrix
    printf("Printing similarity matrix:\n");
    for(int i=0; i<MATRIX_ROW; i++){
        for (int j=0; j<MATRIX_ROW; j++){
            printf("%f ", similarity_matrix[i][j]);
        }
        printf("\n");
    }

}


double entropyCalculation (char data_matrix[][MATRIX_COLUNM][STRING_LENGTH],double similarity_matrix[][MATRIX_ROW],int withoutColum){
    
    if (PRINT_LOGS){
    printf("-----------------------------------------\n");
    printf("BEGIN - function: entropyCalculation for column %d\n", withoutColum);
    printf("-----------------------------------------\n");
    }

    //Matrix type double (*)[MATRIX_ROW]
    //double similarity_matrix[MATRIX_ROW][MATRIX_ROW];

    if (PRINT_LOGS){
    printf("Begin - calculation of the similarity matrix for column %d ...\n\n", withoutColum);
    }

    double sum=0;
    for(int row_s=0; row_s < MATRIX_ROW; row_s++){
        for(int column_s=0; column_s< MATRIX_ROW; column_s++){

            for (int column_d=0; column_d < MATRIX_COLUNM; column_d++){
                if (withoutColum == 0){
                   sum = sum + hammingDist(data_matrix[row_s][column_d],data_matrix[column_s][column_d]);
                }
                else{
                    if (withoutColum-1 != column_d)
                        sum = sum + hammingDist(data_matrix[row_s][column_d],data_matrix[column_s][column_d]);

                }
                
            }
            if (withoutColum == 0)
                similarity_matrix[row_s][column_s]= (double) sum/MATRIX_COLUNM;
            
            else
                similarity_matrix[row_s][column_s]= (double) sum/(MATRIX_COLUNM-1);
            

            sum=0;
        }
    }

    if (PRINT_LOGS_MATRIX){
        printSimilarityMatrix(similarity_matrix);
    }
    if (PRINT_LOGS){
        printf("\nEnd - calculation of the similarity matrix for column %d.\n\n", withoutColum);
        printf("Begin - calculation of entroy for column %d...\n\n", withoutColum);
        }

    double entropy=0;
    for (int i=0; i < MATRIX_ROW-2; i++){
        for (int j=i+1; j < MATRIX_ROW-1;j++){
            if (similarity_matrix[i][j] != 0.0)
               entropy = entropy + ((similarity_matrix[i][j]*log(similarity_matrix[i][j])) +  ((1-similarity_matrix[i][j]) * log(1-similarity_matrix[i][j])));
        }
    }
    entropy = (-1)* entropy;
    if (PRINT_LOGS){
    printf("The entropy value for column %d is: %f\n\n", withoutColum,entropy);
    printf("end - calculation of entroy for column %d.\n", withoutColum);

    printf("-----------------------------------------\n");
    printf("END - function: entropyCalculation for column %d\n", withoutColum);
    printf("-----------------------------------------\n");
    }
    return entropy;
}


int main (){

    printf("****************************************************************************\n");
    printf("    BEGIN - Entropy Calculation for Dimensionality Reduction\n");
    printf("***************************************************************************\n\n");

    // Varibles for mesuered time elapsed
    struct timeval tval_before, tval_after, tval_result;
    gettimeofday(&tval_before, NULL);

    //reading CSV file
    readCSVFile(main_matrix);

    //Array that has all entropy values
    double vaulesOfEntropyWithoutColums [MATRIX_COLUNM+1];

    for (int column = 0; column < MATRIX_COLUNM+1; column++){
        //entropyCalculation(main_matrix,0) - 0 means with all columns
        vaulesOfEntropyWithoutColums[column] = entropyCalculation(main_matrix, similarity_matrix, column);
    }

    printf("\n");
    printf("The entropy values for each column are as follows:\n");
    for (int i = 0; i < MATRIX_COLUNM+1; i++){
        if (i==0) // 0 in is saved the entropy vaule with all the columns
            printf("    The value with all columns    : %f\n",vaulesOfEntropyWithoutColums[i]);

        else
            printf("    The value without the column %d: %f\n",i,vaulesOfEntropyWithoutColums[i]);
    }

    // getting Time elapsed
    gettimeofday(&tval_after, NULL);
    timersub(&tval_after, &tval_before, &tval_result);
    
    printf("\n****************************************************************************\n");
    printf("    END - Entropy Calculation for Dimensionality Reduction\n");
    printf("***************************************************************************\n");
    printf("\nTime elapsed: %ld.%06ld\n", (long int)tval_result.tv_sec, (long int)tval_result.tv_usec);
}



Overwriting main_sequential.c


In [21]:
!gcc main_sequential.c -o main_sequential.o -lm
!./main_sequential.o

****************************************************************************
    BEGIN - Entropy Calculation for Dimensionality Reduction
***************************************************************************


The entropy values for each column are as follows:
    The value with all columns    : 196911.540504
    The value without the column 1: 190800.237213
    The value without the column 2: 190762.940123
    The value without the column 3: 190777.488703
    The value without the column 4: 190532.962731
    The value without the column 5: 190576.118800
    The value without the column 6: 190768.477907
    The value without the column 7: 190743.308596
    The value without the column 8: 190548.023248
    The value without the column 9: 190693.583444
    The value without the column 10: 190608.243396

****************************************************************************
    END - Entropy Calculation for Dimensionality Reduction
*********************************************

## 2. Open MP code

In [ ]:
#@title main_omp.c
%%writefile main_omp.c

/*******************************************************************************
*
* Program: Dimensionality Reduction With Entropy to a data Matrix set
* 
* Description: Project - Distributed Computing
* 
* Universidad Nacional de Colombia
* 
* Author: Javier Eduardo Duarte Aunta
*
* Octuber 2022
*******************************************************************************/

#include <stdio.h>
#include <math.h>
#include <string.h>
#include <sys/time.h>
#include "omp.h"

#define MATRIX_ROW 1000
#define MATRIX_COLUNM 6
#define STRING_LENGTH 4
#define MAX_LEN_LINE STRING_LENGTH*MATRIX_COLUNM

#define PRINT_LOGS 0
#define PRINT_LOGS_MATRIX 0


#define NUM_THREADS 8

//Matrix type double (*)[MATRIX_ROW]
double similarity_matrix[MATRIX_ROW][MATRIX_ROW];

//Matrix type char (*)[MATRIX_COLUNM][STRING_LENGTH]
char main_matrix[MATRIX_ROW][MATRIX_COLUNM][STRING_LENGTH]={/*  */};


// function to calculate Hamming distance
int hammingDist(char* str1, char* str2)
{
    int i = 0, count = 0;
    while (str1[i] != '\0') {
        if (str1[i] == str2[i] && str1[i] != '\n' && str2[i] != '\n')
            count++;
        i++;
    }
    return count;
}

void printDataMatrix(char data_matrix[][MATRIX_COLUNM][STRING_LENGTH]){
    // Print read data
    printf("Printing read matrix:\n");
    for(int i=0; i<MATRIX_ROW; i++){
        for (int j=0; j<MATRIX_COLUNM; j++){
            printf("%s ", data_matrix[i][j]);
        }
        //printf(" %d",i);
        printf("\n");
    }
}

void readCSVFile (char main_matrix[MATRIX_ROW][MATRIX_COLUNM][STRING_LENGTH]){
    
    if (PRINT_LOGS){
    printf("-----------------------------------------\n");
    printf("BEGIN - function: readCSVFile \n");
    printf("-----------------------------------------\n");
    }

   char line[MAX_LEN_LINE];
   char *token;

    FILE *file;
    file = fopen("matrix.txt", "r");


    if (file == NULL){
        printf("Error opening file.\n");
    }

    for (int row = 0; row < MATRIX_ROW; row++){
        fgets(line, MAX_LEN_LINE, file);
        //printf("%s", line);

        token= strtok(line, ",");
        for (int colunm = 0; colunm < MATRIX_COLUNM; colunm++)
        {
            if (colunm == MATRIX_COLUNM-1){
                token= strtok(token, "\n");
                strcpy(main_matrix[row][colunm],token);
            }
            else{
                strcpy(main_matrix[row][colunm],token);
                token= strtok(NULL, ",");
            }

        }
        
    }
    fclose(file);
    if (PRINT_LOGS_MATRIX)
        printDataMatrix(main_matrix);

    if (PRINT_LOGS){   
    printf("-----------------------------------------\n");
    printf("END - function: readCSVFile \n");
    printf("-----------------------------------------\n");
    }
}

void printSimilarityMatrix( double similarity_matrix[MATRIX_ROW][MATRIX_ROW]){

    // Print similarity matrix
    printf("Printing similarity matrix:\n");
    for(int i=0; i<MATRIX_ROW; i++){
        for (int j=0; j<MATRIX_ROW; j++){
            printf("%f ", similarity_matrix[i][j]);
        }
        printf("\n");
    }

}


double entropyCalculation (char data_matrix[][MATRIX_COLUNM][STRING_LENGTH],double similarity_matrix[][MATRIX_ROW],int withoutColum){
    
    if (PRINT_LOGS){
    printf("-----------------------------------------\n");
    printf("BEGIN - function: entropyCalculation for column %d\n", withoutColum);
    printf("-----------------------------------------\n");
    }

    //Matrix type double (*)[MATRIX_ROW]
    //double similarity_matrix[MATRIX_ROW][MATRIX_ROW];

    if (PRINT_LOGS){
    printf("Begin - calculation of the similarity matrix for column %d ...\n\n", withoutColum);
    }
    
    #pragma omp parallel
    {
        double sum=0;
        #pragma omp for collapse(2)
        for(int row_s=0; row_s < MATRIX_ROW; row_s++){
            for(int column_s=0; column_s< MATRIX_ROW; column_s++){

                for (int column_d=0; column_d < MATRIX_COLUNM; column_d++){
                    if (withoutColum == 0){
                    sum = sum + hammingDist(data_matrix[row_s][column_d],data_matrix[column_s][column_d]);
                    }
                    else{
                        if (withoutColum-1 != column_d)
                            sum = sum + hammingDist(data_matrix[row_s][column_d],data_matrix[column_s][column_d]);

                    }
                    
                }
                if (withoutColum == 0)
                    similarity_matrix[row_s][column_s]= (double) sum/MATRIX_COLUNM;
                
                else
                    similarity_matrix[row_s][column_s]= (double) sum/(MATRIX_COLUNM-1);
                

                sum=0;
            }
        }
    }

    if (PRINT_LOGS_MATRIX){
        printSimilarityMatrix(similarity_matrix);
    }
    if (PRINT_LOGS){
        printf("\nEnd - calculation of the similarity matrix for column %d.\n\n", withoutColum);
        printf("Begin - calculation of entroy for column %d...\n\n", withoutColum);
        }

    double entropy=0;
 
    #pragma omp parallel for reduction(+:entropy)
    for (int i=0; i < MATRIX_ROW-2; i++){
        for (int j=i+1; j < MATRIX_ROW-1;j++){
            if (similarity_matrix[i][j] != 0.0)
            entropy = entropy + ((similarity_matrix[i][j]*log(similarity_matrix[i][j])) +  ((1-similarity_matrix[i][j]) * log(1-similarity_matrix[i][j])));
        }
    }
    
    entropy = (-1)* entropy;
    if (PRINT_LOGS){
    printf("The entropy value for column %d is: %f\n\n", withoutColum,entropy);
    printf("end - calculation of entroy for column %d.\n", withoutColum);

    printf("-----------------------------------------\n");
    printf("END - function: entropyCalculation for column %d\n", withoutColum);
    printf("-----------------------------------------\n");
    }
    return entropy;
}


int main (){
    printf("****************************************************************************\n");
    printf("    BEGIN - Entropy Calculation for Dimensionality Reduction with OMP\n");
    printf("***************************************************************************\n\n");

    omp_set_num_threads(NUM_THREADS);

    // Varibles for mesuered time elapsed
    struct timeval tval_before, tval_after, tval_result;
    gettimeofday(&tval_before, NULL);

    //reading CSV file
    readCSVFile(main_matrix);

    //Array that has all entropy values
    double vaulesOfEntropyWithoutColums [MATRIX_COLUNM+1];

    printf("Number of processors available = %d\n", omp_get_num_procs ( ) );
    printf( "Number of threads =              %d\n", omp_get_max_threads ( ) );
    printf("\n");
 
    for (int column = 0; column < MATRIX_COLUNM+1; column++){
        //entropyCalculation(main_matrix,0) - 0 means with all columns
        vaulesOfEntropyWithoutColums[column] = entropyCalculation(main_matrix, similarity_matrix, column);
    }
    printf("\n");
    printf("The entropy values for each column are as follows:\n");
    for (int i = 0; i < MATRIX_COLUNM+1; i++){
        if (i==0) // 0 in is saved the entropy vaule with all the columns
            printf("    The value with all columns    : %f\n",vaulesOfEntropyWithoutColums[i]);

        else
            printf("    The value without the column %d: %f\n",i,vaulesOfEntropyWithoutColums[i]);
    }
    // getting Time elapsed
    gettimeofday(&tval_after, NULL);
    timersub(&tval_after, &tval_before, &tval_result);

    printf("\n****************************************************************************\n");
    printf("    END - Entropy Calculation for Dimensionality Reduction with OMP\n");
    printf("***************************************************************************\n");
    printf("\nTime elapsed: %ld.%06ld\n", (long int)tval_result.tv_sec, (long int)tval_result.tv_usec);
}


Writing main_omp.c


In [ ]:
!gcc main_omp.c -o main_omp.o -fopenmp -lm
!./main_omp.o

****************************************************************************
    BEGIN - Entropy Calculation for Dimensionality Reduction with OMP
***************************************************************************

Number of processors available = 2
Number of threads =              8


The entropy values for each column are as follows:
    The value with all columns    : 25412.882898
    The value without the column 1: 23572.942240
    The value without the column 2: 23560.102742
    The value without the column 3: 23589.488726
    The value without the column 4: 23559.466225
    The value without the column 5: 23671.603712
    The value without the column 6: 23600.152361

****************************************************************************
    END - Entropy Calculation for Dimensionality Reduction with OMP
***************************************************************************

Time elapsed: 0.300863


## 3. CUDA code

In [ ]:
#@title Query CUDA

# Query the device to know which GPU is being used
!nvcc --version
%cd /usr/local/cuda-11.2/samples/1_Utilities/deviceQuery
!make
!./deviceQuery

In [ ]:
%cd /content/
!pwd

/content
/content


In [22]:
#@title main_cuda.cu
%%writefile main_cuda.cu
/*******************************************************************************
*
* Program: Dimensionality Reduction With Entropy to a data Matrix set
* 
* Description: Project - Distributed Computing
* 
* Universidad Nacional de Colombia
* 
* Author: Javier Eduardo Duarte Aunta
*
* Octuber 2022
*******************************************************************************/

#include <stdio.h>
#include <math.h>
#include <string.h>
#include <sys/time.h>

// For the CUDA runtime routines (prefixed with "cuda_")
#include <cuda_runtime.h>

#define MATRIX_ROW 2560
#define MATRIX_COLUNM 10
#define STRING_LENGTH 4
#define MAX_LEN_LINE STRING_LENGTH*MATRIX_COLUNM

#define PRINT_LOGS 0

#define BLOCKSPERGRID  40
#define NUMTHREADS 64


//Matrix type char (*)[MATRIX_COLUNM][STRING_LENGTH]
char main_matrix[MATRIX_ROW][MATRIX_COLUNM][STRING_LENGTH]={/*  */};

//Matrix type double (*)[MATRIX_ROW]
double similarity_matrix[MATRIX_ROW][MATRIX_ROW]={0.0};

// function to calculate Hamming distance
__device__ int hammingDist_CUDA (char* str1, char* str2)
{
    int i = 0, count = 0;
    while (str1[i] != '\0') {
        if (str1[i] == str2[i] && str1[i] != '\n' && str2[i] != '\n')
            count++;
        i++;
    }
    return count;
}


void readCSVFile (char main_matrix[MATRIX_ROW][MATRIX_COLUNM][STRING_LENGTH])
{
    
    if (PRINT_LOGS){
    printf("-----------------------------------------\n");
    printf("BEGIN - function: readCSVFile \n");
    printf("-----------------------------------------\n");
    }

   char line[MAX_LEN_LINE];
   char *token;

    FILE *file;
    file = fopen("matrix.txt", "r");


    if (file == NULL){
        printf("Error opening file.\n");
    }

    for (int row = 0; row < MATRIX_ROW; row++){
        fgets(line, MAX_LEN_LINE, file);
        //printf("%s", line);

        token= strtok(line, ",");
        for (int colunm = 0; colunm < MATRIX_COLUNM; colunm++)
        {
            if (colunm == MATRIX_COLUNM-1){
                token= strtok(token, "\n");
                strcpy(main_matrix[row][colunm],token);
            }
            else{
                strcpy(main_matrix[row][colunm],token);
                token= strtok(NULL, ",");
            }

        }
        
    }
    fclose(file);

    if (PRINT_LOGS){   
    printf("-----------------------------------------\n");
    printf("END - function: readCSVFile \n");
    printf("-----------------------------------------\n");
    }
}


__global__ void similarityMatrixCalculation_CUDA (char *data_matrix, double *similarity_matrix, int withoutColum)
{
    int index_row = (blockDim.x * blockIdx.x) + threadIdx.x;

    double sum=0;

    //for(int row_s=0; row_s < MATRIX_ROW; row_s++){

    for(int column_s=0; column_s< MATRIX_ROW; column_s++){

        for (int column_d=0; column_d < MATRIX_COLUNM; column_d++){
            if (withoutColum == 0){

                sum = sum + hammingDist_CUDA(&data_matrix[index_row*MATRIX_COLUNM*STRING_LENGTH + column_d*STRING_LENGTH], &data_matrix[column_s*MATRIX_COLUNM*STRING_LENGTH + column_d*STRING_LENGTH]);
                //sum = sum + hammingDist_CUDA(data_matrix[row_s][column_d],data_matrix[column_s][column_d]);
            }
            else{
                if (withoutColum-1 != column_d)
                    sum = sum + hammingDist_CUDA(&data_matrix[index_row*MATRIX_COLUNM*STRING_LENGTH + column_d*STRING_LENGTH], &data_matrix[column_s*MATRIX_COLUNM*STRING_LENGTH + column_d*STRING_LENGTH]);
                    //sum = sum + hammingDist_CUDA(data_matrix[row_s][column_d],data_matrix[column_s][column_d]);

            }
            
        }

        if (withoutColum == 0){
            similarity_matrix[index_row*MATRIX_ROW + column_s]= (double) sum/MATRIX_COLUNM;
            //similarity_matrix[index_row][column_s]= (double) sum/MATRIX_COLUNM;
        }


        
        else{
            similarity_matrix[index_row*MATRIX_ROW + column_s]= (double) sum/(MATRIX_COLUNM-1);
            //similarity_matrix[index_row][column_s]= (double) sum/(MATRIX_COLUNM-1);
        }
        sum=0;
    }

    ////}
}

void entropyCalculation(double similarity_matrix[MATRIX_ROW][MATRIX_ROW], double final_entropy[MATRIX_ROW], int withoutColum){

    double entropy=0;
    for (int i=0; i < MATRIX_ROW-2; i++){
        for (int j=i+1; j < MATRIX_ROW-1;j++){
            if (similarity_matrix[i][j] != 0.0)
            entropy = entropy + ((similarity_matrix[i][j]*log(similarity_matrix[i][j])) +  ((1-similarity_matrix[i][j]) * log(1-similarity_matrix[i][j])));
        }
    }
    entropy = (-1) * entropy;
    final_entropy[withoutColum] = entropy;
}

int main ()
{

    printf("****************************************************************************\n");
    printf("    BEGIN - Entropy Calculation for Dimensionality Reduction with CUDA\n");
    printf("***************************************************************************\n\n");

    // Varibles for mesuered time elapsed
    struct timeval tval_before, tval_after, tval_result;
    gettimeofday(&tval_before, NULL);

    //reading CSV file
    readCSVFile(main_matrix);

    //*******************************************************
    // BEGIN CUDA CODE
    //*******************************************************

    
    // Flating main matrix AND similarity matrix
    // varibles in host
    char *h_main_matrix = (char *) main_matrix;
    double *h_similarity_matrix = (double *) similarity_matrix;

    //Array that has all entropy values
    double h_entropy[MATRIX_COLUNM+1];

    // varibles in device
    char *d_main_matrix;
    double *d_similarity_matrix;
    double *d_entropy;

    //getting size of each varible
    int size_main_matrix = sizeof(main_matrix);
    int size_similarity_matrix = sizeof(double)*MATRIX_ROW*MATRIX_ROW;
    int size_entropy = sizeof(h_entropy);

 
    //---------------------------
    // BEGIN - ALLOCATE IN DEVICE
    //---------------------------

    cudaError_t err = cudaSuccess;

    //For main_matrix
    cudaMalloc( (void **) &d_main_matrix, size_main_matrix);
    if (err != cudaSuccess){
        fprintf(stderr, "Failed to allocate d_main_matrix in device (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    //For similarity_matrix
    cudaMalloc( (void **) &d_similarity_matrix, size_similarity_matrix);
    if (err != cudaSuccess){
        fprintf(stderr, "Failed to allocate d_similarity_matrix in device (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    //For h_entropy
    cudaMalloc( (void **) &d_entropy, size_entropy);
    if (err != cudaSuccess){
        fprintf(stderr, "Failed to allocate d_entropy in device (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    //---------------------------
    // END - ALLOCATE IN DEVICE
    //---------------------------


    //---------------------------
    // BEGIN - COPY VAULE TO DEVICE
    //---------------------------

    //For main_matrix
    err = cudaMemcpy(d_main_matrix, h_main_matrix, size_main_matrix, cudaMemcpyHostToDevice);
    if (err != cudaSuccess){
        fprintf(stderr, "Failed to copy vector h_main_matrix from host to device (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    //For similarity_matrix

    //For h_entropy

    //---------------------------
    // END  - COPY VAULE TO DEVICE
    //---------------------------


    //---------------------------
    // BEGIN - LAUNCH KERNEL
    //---------------------------
    // Launch kernel N times (numbers columns) 

    for (int column = 0; column < MATRIX_COLUNM+1; column++){
    similarityMatrixCalculation_CUDA<<<NUMTHREADS,BLOCKSPERGRID>>>(d_main_matrix, d_similarity_matrix, column);

        //---------------------------
        // BEGIN - COPY VAULES TO HOST
        //---------------------------
        //h_similarity_matrix
        err = cudaMemcpy(h_similarity_matrix, d_similarity_matrix, size_similarity_matrix, cudaMemcpyDeviceToHost);
        if (err != cudaSuccess){
            fprintf(stderr, "Failed to copy vector h_similarity_matrix from device to host (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        entropyCalculation(similarity_matrix, h_entropy, column);
        //---------------------------
        // END -  COPY VAULES TO HOST
        //---------------------------
    }
    //---------------------------
    // END -  LAUNCH KERNEL
    //---------------------------


    //---------------------------
    // BEGIN - PRINT DATA FROM GPU
    //---------------------------

    //For h_entropy
    printf("\n");
    printf("The entropy values for each column are as follows:\n");
    for (int i = 0; i < MATRIX_COLUNM+1; i++){
        if (i==0) // 0 in is saved the entropy vaule with all the columns
            printf("    The value with all columns    : %f\n",h_entropy[i]);

        else
            printf("    The value without the column %d: %f\n",i,h_entropy[i]);
    }

    //---------------------------
    // END -  PRINT DATA FROM GPU
    //---------------------------


    //---------------------------
    // BEGIN - FREE MEMORY 
    //---------------------------

    //For d_main_matrix
    err = cudaFree(d_main_matrix);
    if (err != cudaSuccess){
        fprintf(stderr, "Failed to free device vector d_main_matrix (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    //For similarity_matrix
    err = cudaFree(d_similarity_matrix);
    if (err != cudaSuccess){
        fprintf(stderr, "Failed to free device vector d_similarity_matrix (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    //For h_entropy
    err = cudaFree(d_entropy);
    if (err != cudaSuccess){
        fprintf(stderr, "Failed to free device vector d_entropy (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    //---------------------------
    // END - FREE MEMORY
    //---------------------------

    //*******************************************************
    // END - CUDA CODE
    //*******************************************************

    // getting Time elapsed
    gettimeofday(&tval_after, NULL);
    timersub(&tval_after, &tval_before, &tval_result);
    
    printf("\n****************************************************************************\n");
    printf("    END - Entropy Calculation for Dimensionality Reduction with CUDA\n");
    printf("***************************************************************************\n");
    printf("\nTime elapsed: %ld.%06ld\n", (long int)tval_result.tv_sec, (long int)tval_result.tv_usec);
    return 0;
}


Overwriting main_cuda.cu


In [23]:
!nvcc main_cuda.cu -o main_cuda.o 

In [24]:
!./main_cuda.o

****************************************************************************
    BEGIN - Entropy Calculation for Dimensionality Reduction with CUDA
***************************************************************************


The entropy values for each column are as follows:
    The value with all columns    : 196911.540504
    The value without the column 1: 190800.237213
    The value without the column 2: 190762.940123
    The value without the column 3: 190777.488703
    The value without the column 4: 190532.962731
    The value without the column 5: 190576.118800
    The value without the column 6: 190768.477907
    The value without the column 7: 190743.308596
    The value without the column 8: 190548.023248
    The value without the column 9: 190693.583444
    The value without the column 10: 190608.243396

****************************************************************************
    END - Entropy Calculation for Dimensionality Reduction with CUDA
*************************